Leon 1-1

Name topics
* sort tweets by highest coefficient for each topic
* look thru these tweets to determine topic name

* or have one column with the topic (being the highest coefficient)
    * can use this column to create a hue
    * bar charts ok

* no need to rearrange words

add bot_proba & determine threshold
* or, keep bot_proba & show avg bot_proba per topic!!!
    * box & whiskers

pull statistics
* x% of candidate ys tweets were bots
    * per topic
* what were bots tweeting/not tweeting about

In [1]:
import pickle

import pandas as pd
import numpy as np

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF, LatentDirichletAllocation

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
from nltk.corpus import stopwords as nltk_stop
from stop_words import stop_words as custom_stop

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', None)

In [4]:
data = pd.read_pickle("pickle/n2tk_limited.pick")
data.shape

(363310, 11)

In [5]:
data.columns

Index(['id', 'date', 'time', 'user_id', 'username', 'hashtags', 'trump',
       'biden', 'original', 'tweet', 'num_tokens'],
      dtype='object')

In [6]:
with open("pickle/nmf_model.pick", 'rb') as f:
    nmf_model = pickle.load( f)

with open("pickle/nmf_doc_topics.pick", 'rb') as f:
    doc_topic = pickle.load( f)

with open("pickle/topic_words.pick", 'rb') as f:
    topic_words = pickle.load( f)

In [7]:
doc_topic.shape

(363310, 13)

## Naming Topics

Before we can name our topics, we need to classify each tweet in our original DataFrame with its corresponding topic.

The values in our doc_topic matrix has rows=tweets, columns=topics, vaues=coefficients. We'll simply classify each tweet with the largest coefficient.

In [8]:
topic = doc_topic.argmax(axis=1)
data['tweet_topic'] = topic

In [9]:
# nmf_model.components_

In [10]:
# doc_topic

In [11]:
data.shape

(363310, 12)

In [12]:
doctop = pd.DataFrame(doc_topic)
doctop

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000836,0.000300,0.000000,0.000892,0.000000,0.000066,0.000000,0.000729,0.000038,0.003112,0.003755,0.000000,0.000308
1,0.003752,0.001583,0.000750,0.000000,0.000577,0.000377,0.000000,0.006958,0.001502,0.000000,0.000000,0.000266,0.001017
2,0.001226,0.001190,0.000000,0.000064,0.047775,0.000000,0.000047,0.006574,0.000035,0.000000,0.000134,0.000000,0.000000
3,0.001565,0.001057,0.002666,0.049943,0.000000,0.003429,0.000857,0.000000,0.000000,0.004633,0.002868,0.000472,0.000000
4,0.000000,0.028202,0.000000,0.000000,0.000000,0.000222,0.000520,0.000000,0.000000,0.000000,0.020264,0.038826,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363305,0.004147,0.001326,0.000000,0.000000,0.002541,0.000889,0.000000,0.010283,0.001982,0.001314,0.005024,0.000000,0.002434
363306,0.002246,0.000000,0.000000,0.000141,0.000198,0.000482,0.000055,0.000000,0.055385,0.029034,0.000036,0.000000,0.000688
363307,0.002394,0.000766,0.007797,0.001437,0.000000,0.002167,0.001311,0.000000,0.000000,0.002420,0.001564,0.002703,0.006103
363308,0.011595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's sort our tweets by topic coefficient, so that we can see which tweets have the highest coefficient for each topic. This will help us to identify the meaning of each topic.

Let's get the top ten tweets per topic.

In [13]:
# ensure topics start at 1 (and not zero)
if list(doctop.columns)[0] == 0:
    doctop.columns = doctop.columns + 1
    data.tweet_topic = data.tweet_topic + 1

# read topic tweets by updating topic_num
topic_num = 9
top10_ix = doctop[topic_num].sort_values(ascending=False).head(100).index # returns indexes of best tweets matching topic 1
data[['username', 'tweet', 'original']].iloc[top10_ix]


,username,tweet,original
576009,yungmarc2,joebiden unrestrained roast campaign barackobama time rally donaldtrump,"Los Angeles Times: Obama, unrestrained, roasts Trump at Biden campaign rallies. https://t.co/tLpDr8jXHv via @GoogleNews"
700053,robjt265,joebiden mala campaign ka sank quixotic mistake,@JoeBiden Make no mistake: Ka’mala sank your quixotic campaign on Wednesday night. #TrojanJoe #HeelsUp #ObamaGate
188590,chloechloe616,joebiden campaign w ad covid dead donaldtrump,"@SymoneDSanders don't back off on the ads and campaign. Trump campaign isn't. We need Joe to win. Trump has already minimized covid w/ over 200,000 dead Americans. We can't take anymore Trump"
462569,catseyes00,joebiden weve onwards campaign spiral downward donaldtrump,"@AndrewSolender @JoeBiden @TimMurtaugh From this point onwards, everything the Trump campaign does is going to be self-damaging. We've now entered the downward spiral..."
155519,dollym1852,negative joebiden need campaign ad forward donaldtrump,@fred_guttenberg @joncoopertweets @JoeBiden Trump. Campaign is NOT pulling Negative ADs. Biden needs to go forward with ALL ads.
679303,trendsnewsday,speech joebiden fiery counterpoint campaign barackobama make,"Barack Obama makes fiery 2020 campaign speech, counterpoint to low-key Biden https://t.co/jJC5xb6fhI"
563175,rogue_corq,worrisome joebiden campaign lead democrat hillaryclinton donaldtrump,"Biden leads Trump. So did Hillary Clinton. For Democrats, it’s a worrisome campaign deja vu https://t.co/ZPl728Q6ls"
126043,shining54199628,cnn joebiden heather campaign work name donaldtrump,"@RWTrollPatrol @realDonaldTrump @CNN @nytimes I’ve seen her before. She works for the Biden campaign in Iowa. Her name isn’t Monica, it’s Heather."
711575,plutonetworks,account money campaign company know want donaldtrump,"@maggieNYT Trump is trying to keep his campaign money now, them money laundering to Trump companies accounts... He knows he will lose and wants to keep the money."
466100,susanc916,negative joebiden donaldtrump usual campaign ad say,"@mmpadellan @Cctenpro1 The Trump campaign says they’re not pulling their negative ads like Biden pulled his, so it appears to be campaigning as usual."


## Final Topic Names

After looking through the topic_words.csv file and testing each of the topics in the above cell, here's what I came up with:

1. **hunter-biden**
2. **pro-vote**
3. **covid-stats**
4. **vp-candidates**
5. **debate**
6. **trump-covid-rally**
7. **past-candidates**
8. **polls**
9. **negative-campaign-ads**
11. **foreign-affairs**
12. **racism**
13. **truth & lies**
14. **covid-restrictions**


In [14]:
data['topic'] = doc_topic.argmax(axis=1)
if np.min(data.topic.to_numpy()) == 0:
    data['topic'] += 1
    
topics = {
    1:'hunter_biden',
    2:'pro_vote',
    3:'covid_stats',
    4:'vp_candidates',
    5:'debate',
    6:'trump_covid_rally',
    7:'past_candidates',
    8:'polls',
    9:'negative_campaign_ads',
    10: 'foreign_affairs',
    11: 'racism',
    12: 'truth_and_lies',
    13: 'covid_restrictions',
}

data['topic'] = data['topic'].map(topics)
data.to_pickle("pickle/tweet_topics.pick")

In [15]:
data[['username', 'date', 'topic', 'original', 'tweet']].sample(10)

,username,date,topic,original,tweet
775306,clairenj_,2020-11-02,vp_candidates,Abortion is child sacrifice. If you vote democrat you are supporting child sacrifice. @JoeBiden @KamalaHarris @HillaryClinton #AbortionIsMurder #voteforlife #SaveTheChildren,joebiden vote child abortion supporting sacrifice kamalaharris hillaryclinton
545116,terence,2020-10-26,hunter_biden,Saying “We must treat the Hunter Biden leaks as if they were a foreign intelligence operation — even if they probably aren’t.” = Saying “We must treat John Paul MacIsaac as if he were a foreign intelligence operative — even if he probably isn’t.” 🙁🤦 https://t.co/twYad3nbj9,treat probably hunterbiden leak operation foreign operative intelligence
276038,rixtrucker,2020-10-13,hunter_biden,"@TheRickWilson seen the,,60 minutes special they did on you guys I'm dem and just wanted to say thanks and keep up the fight kick trump to the curb appreciate everything you guys do",curb guy kick appreciate minute fight special donaldtrump
618079,f_marygeorge,2020-10-22,hunter_biden,"There's a difference between criminals and crooks. Crooks steal. Criminals blow some guy's brains out. So, how do you classify TRUMP??",crook guy difference brain criminal blow there steal donaldtrump
904040,scotirishchick,2020-11-02,hunter_biden,@donwinslow @FoxNews And they’re the reason our country is so divided. I know way too many Fox News viewers &amp; Trump supporters. They are one in the same. Our country will continue to be divided &amp; in turmoil as long as Fox News exists. They need to be shut down.,foxnews trumpsupporter continue shut divided viewer turmoil
598248,housekyrze,2020-10-24,foreign_affairs,"@JoeBiden Sir, next time people want to give you grief about oil, just remind them that even the Saudis are looking to the future &amp; shifting their economy away from oil &amp; have invested 3/4 Trillion $ in the last 5yrs towards the tech industry.",grief joebiden oil tech shifting industry away economy towards future remind trillion
476777,casiewestman,2020-10-27,covid_stats,“B” &amp; “H” conspired with China to begin the Coronavirus in order to blame it on Trump. This is why Biden didn’t want to close the border. He knew exactly what was about to happen! China wants Biden in office bad! Think about it. They have been working together a long time.,joebiden together begin h exactly working china b close covid border blame want donaldtrump
706053,stelthesizzler,2020-10-27,racism,@LetsWorkTogeth3 @MihaTaa @Prometheus19782 @larryelder i’m not even being hostile i’m pointing out the obvious. she openly supports trump and the things he said about POC are so rude and racist. i’m not assuming what she meant i KNOW what she meant. but continue to not see the obvious with closed eyes.,pointing assuming obvious continue closed support thing eye rude racist hostile openly meant donaldtrump
141531,natemax,2020-10-05,truth_and_lies,"@Idliva This man has shown a callous disregard for his health, his employees' health and the health and safety of all Americans. It's time for @JoeBiden to run all his ""attack"" ads that simply tell the truth. https://t.co/IhMj3U7WJL",truth disregard employee joebiden shown simply safety ad callous attack health
801977,kthomasellis,2020-10-18,hunter_biden,"When I think about it, Trump has never been the president of this country. He has never represented the whole country. As a result he divided the country worse than ever. He doesn't know what a president is supposed to do. He made enemies of us. Yes, I hate him.",u result worse supposed whole divided enemy hate donaldtrump
